# Install and Import Necessary Dependencies

In [ ]:
!pip install torch
!pip install transformers pandas numpy

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

# Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Load Data

Data obtained from https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch?resource=download





In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
import os
import pandas as pd

In [10]:
# Change to correct directory
try:
  os.chdir("/content/drive/MyDrive/Personal Projects/Suicide Detection Dataset")
  print('Directory change success')
except OSError:
  print('Directory change failed')

Directory change success


In [14]:
# Load dataset
df = pd.read_csv('Suicide_Detection.csv')
df.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


# Preprocess Data

Why? 🤔

Preprocessing standardizes the text so that computer models can better understand and work with human input. It also reduces computational complexity when model training. We want to avoid scenerios where words such as "Game" and "game" are counted as 2 different words.

Tasks:
- Fix spelling errors
- Change all letters to lowercase
- Remove stop words (words that do not contribute to the overall meaning of the text)
- Expand contractions (I've ➡ I have)
- Remove extra whitespaces
- Remove accents, URLs, special characters (including Unicode characters)




In [ ]:
!pip install unidecode contractions pyspellchecker wordninja symspellpy

In [ ]:
import numpy as np
import nltk
import unidecode
import contractions as contract
import wordninja
from spellchecker import SpellChecker
from symspellpy import SymSpell, Verbosity

nltk.download('stopwords')

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

In [50]:
# Defining methods

# Spell check and change to lowercase with Symspell
def fix_spelling(text):
  suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
  corrected_text = suggestions[0].term
  return corrected_text

def remove_stop_words(text):
  stop_words = stopwords.words('english')
  selected_stop_words = [word for word in stop_words if word not in ['no', 'not']]
  return ' '.join([word for word in text.split(' ') if word not in stop_words])
